# Naive algorithm 
This algorithm only checks if a log key pattern has been observed before or not.

In [1]:
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings('ignore')

In [2]:
import datetime
from datetime import timezone

try:
    import sys
    sys.path.append("../dataset/")
    from dataset_producer import DatasetProducer, DATASETS, get_log_pattern_key, get_log_timestamp
    iwiget=True
except:
    from dataset.dataset_producer import DatasetProducer, DATASETS, get_log_pattern_key, get_log_timestamp
    iwiget=False

In [3]:
SUBMIT_LIMIT = datetime.timedelta(minutes=1)


def naive(ds, iwiget):
    seen_pattern_keys = set()
    print("Learning phase")
    for log_list in ds.emulate_log_sequence(iwiget=iwiget, training=True):
        log = log_list[-1]
        log_pattern_key = get_log_pattern_key(log)
        seen_pattern_keys.add(log_pattern_key)
    
    print("Testing phase")
    last_submit = datetime.datetime(1970, 1, 1, tzinfo=timezone.utc)
    for log_list in ds.emulate_log_sequence(iwiget=iwiget): 
        log = log_list[-1]
        log_pattern_key = get_log_pattern_key(log)
        log_timestamp = get_log_timestamp(log)
        if log_pattern_key not in seen_pattern_keys and log_timestamp - last_submit > SUBMIT_LIMIT:
            dataset.submit(log["@timestamp"])
            last_submit = log_timestamp

In [4]:
dataset = DatasetProducer(DATASETS.DATASET_TEST, window=1)
naive(dataset, iwiget)
dataset.evaluate()

Learning phase


IntProgress(value=0, description='Files:', max=2)

IntProgress(value=0, description='Segments:', max=10)

Testing phase


IntProgress(value=0, description='Files:', max=2)

IntProgress(value=0, description='Segments:', max=10)

_____________________________________________________
Problems detected: 1 of 1
FP count: 0
Maintenance hits: 1
Total score: 0
_____________________________________________________


In [5]:
dataset = DatasetProducer(DATASETS.DATASET_1, window=1)
naive(dataset, iwiget)
dataset.evaluate()
pickle.dump( dataset.evaluation, open( "dumps/naive_dataset_1.p", "wb" ) )

Learning phase


IntProgress(value=0, description='Files:', max=97)

IntProgress(value=0, description='Segments:', max=10)

Testing phase


IntProgress(value=0, description='Files:', max=407)

IntProgress(value=0, description='Segments:', max=10)

In [6]:
dataset = DatasetProducer(DATASETS.DATASET_2, window=1)
naive(dataset, iwiget)
dataset.evaluate()
pickle.dump( dataset.evaluation, open( "dumps/naive_dataset_2.p", "wb" ) )

Learning phase


[autoreload of dataset_producer failed: Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
RecursionError: maximum recursion depth exceeded
]


IntProgress(value=0, description='Files:', max=73)

IntProgress(value=0, description='Segments:', max=10)

Testing phase


IntProgress(value=0, description='Files:', max=71)

IntProgress(value=0, description='Segments:', max=10)